In [1]:
import numpy as np

# Activation function: sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid function
def sigmoid_derivative(x):
    return x * (1 - x)

# Loss function: Mean Squared Error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# RNN class definition
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights
        self.U = np.random.randn(hidden_size, input_size)  # Input to hidden
        self.W = np.random.randn(hidden_size, hidden_size) # Hidden to hidden
        self.V = np.random.randn(output_size, hidden_size) # Hidden to output

        # Initialize biases
        self.b = np.zeros((hidden_size, 1))
        self.c = np.zeros((output_size, 1))

        # Hidden state initialization
        self.hidden_state = np.zeros((hidden_size, 1))

    def forward(self, x):
        # Forward pass for each time step
        self.hidden_state = sigmoid(np.dot(self.U, x) + np.dot(self.W, self.hidden_state) + self.b)
        y_pred = sigmoid(np.dot(self.V, self.hidden_state) + self.c)
        return y_pred

    def backward(self, x, y_true, y_pred, learning_rate=0.01):
        # Compute output error
        output_error = y_pred - y_true
        output_delta = output_error * sigmoid_derivative(y_pred)

        # Compute hidden error
        hidden_error = np.dot(self.V.T, output_delta)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden_state)

        # Update weights and biases
        self.V -= learning_rate * np.dot(output_delta, self.hidden_state.T)
        self.U -= learning_rate * np.dot(hidden_delta, x.T)
        self.W -= learning_rate * np.dot(hidden_delta, self.hidden_state.T)
        self.b -= learning_rate * hidden_delta
        self.c -= learning_rate * output_delta

    def train(self, X, Y, epochs=1000, learning_rate=0.01):
        for epoch in range(epochs):
            epoch_loss = 0
            for i in range(len(X)):
                # Forward pass
                x = X[i].reshape(-1, 1)
                y_true = Y[i].reshape(-1, 1)
                y_pred = self.forward(x)

                # Compute loss
                epoch_loss += mean_squared_error(y_true, y_pred)

                # Backward pass
                self.backward(x, y_true, y_pred, learning_rate)

            # Print the loss for every 100th epoch
            if epoch % 100 == 0:
                print(f'Epoch {epoch}/{epochs}, Loss: {epoch_loss / len(X)}')

    def predict(self, X):
        predictions = []
        for i in range(len(X)):
            x = X[i].reshape(-1, 1)
            y_pred = self.forward(x)
            predictions.append(y_pred.flatten())
        return np.array(predictions)

# Sample data for demonstration
# Input data (X): 3 samples, each with 2 features
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0]
])

# Target data (Y): 3 samples, each with 1 output
Y = np.array([
    [0],
    [1],
    [1]
])

# Instantiate and train the RNN
input_size = 2
hidden_size = 4
output_size = 1
rnn = SimpleRNN(input_size, hidden_size, output_size)

# Train the RNN
rnn.train(X, Y, epochs=1000, learning_rate=0.01)

# Predict
predictions = rnn.predict(X)
print("\nPredictions:")
print(predictions)


Epoch 0/1000, Loss: 0.28983727665951137
Epoch 100/1000, Loss: 0.23855701020834816
Epoch 200/1000, Loss: 0.22885736759293937
Epoch 300/1000, Loss: 0.22234660747756083
Epoch 400/1000, Loss: 0.2175194630021878
Epoch 500/1000, Loss: 0.21359195325049182
Epoch 600/1000, Loss: 0.21015185135650294
Epoch 700/1000, Loss: 0.2069774736517839
Epoch 800/1000, Loss: 0.20394643187196015
Epoch 900/1000, Loss: 0.20098928975323774

Predictions:
[[0.57488683]
 [0.56142257]
 [0.73405938]]
Epoch 0/1000, Loss: 0.28983727665951137
Epoch 100/1000, Loss: 0.23855701020834816
Epoch 200/1000, Loss: 0.22885736759293937
Epoch 300/1000, Loss: 0.22234660747756083
Epoch 400/1000, Loss: 0.2175194630021878
Epoch 500/1000, Loss: 0.21359195325049182
Epoch 600/1000, Loss: 0.21015185135650294
Epoch 700/1000, Loss: 0.2069774736517839
Epoch 800/1000, Loss: 0.20394643187196015
Epoch 900/1000, Loss: 0.20098928975323774

Predictions:
[[0.57488683]
 [0.56142257]
 [0.73405938]]
